决赛队伍次数统计

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("local[*]").getOrCreate()
summary = spark.read.format("csv").option("header",True).option("encoding","utf-8").option("sep",",").load("../dataset/WorldCupsSummary.csv")

In [2]:
summary.createTempView("summary_tb")

In [3]:
summary.printSchema()

root
 |-- Year: string (nullable = true)
 |-- HostCountry: string (nullable = true)
 |-- Winner: string (nullable = true)
 |-- Second: string (nullable = true)
 |-- Third: string (nullable = true)
 |-- Fourth: string (nullable = true)
 |-- GoalsScored: string (nullable = true)
 |-- QualifiedTeams: string (nullable = true)
 |-- MatchesPlayed: string (nullable = true)
 |-- Attendance: string (nullable = true)
 |-- HostContinent: string (nullable = true)
 |-- WinnerContinent: string (nullable = true)


In [4]:
finalNumber = spark.sql('''
SELECT country,SUM(counts) AS  final_number FROM 
(SELECT Winner AS country,COUNT(Winner) AS counts FROM summary_tb GROUP BY country
UNION ALL
SELECT Second AS country,COUNT(Second) AS counts FROM summary_tb GROUP BY country)  GROUP BY country ORDER BY final_number DESC 
''')

In [5]:
finalNumber.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/worldcup") \
    .option("dbtable", "finalNumber") \
    .option("user", "root") \
    .option("password", "") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .mode("append") \
    .save()

In [6]:
finalNumber.show()

+--------------+------------+
|       country|final_number|
+--------------+------------+
|        Brazil|           7|
|         Italy|           6|
|    Germany FR|           6|
|     Argentina|           5|
|        France|           3|
|   Netherlands|           3|
|       Germany|           2|
|       Uruguay|           2|
|Czechoslovakia|           2|
|       Hungary|           2|
|         Spain|           1|
|       England|           1|
|        Sweden|           1|
|       Croatia|           1|
+--------------+------------+
